In [1]:
!pip install langchain langchain-openai chromadb renumics-spotlight

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.6/252.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
%env OPENAI_API_VERSION=2024-02-15-preview
from google.colab import userdata
import os
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZOAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZOAI_ENDPOINT')

env: OPENAI_API_VERSION=2024-02-15-preview


In [10]:
# helper functions for caching expensive LMM Calls
from pathlib import Path
import pandas as pd
import json
import hashlib


def write_dict_to_file(data_dict, filename):
    """write a dictionary as a json line to a file - allowing for appending"""
    with open(filename, "a") as f:
        f.write(json.dumps(data_dict) + "\n")


def read_dicts_from_file(filename):
    """Read a json line file as a generator of dictionaries - allowing to load multiple dictionaries as list."""
    with open(filename, "r") as f:
        for line in f:
            yield json.loads(line)


def add_cached_column_from_file(df, file_name, merge_on, column):
    """Read a file with cached list of dicts data write it to a dataframe."""

    if Path(file_name).exists():

        cached_answer_correctness = (
            pd.DataFrame(list(read_dicts_from_file(file_name)))
            .drop_duplicates(
                subset=[merge_on],
            )[[column, merge_on]]
            .dropna()
            .reset_index(drop=True)
        )
        return df.merge(
            cached_answer_correctness,
            on=merge_on,
            how="left",
        ).reset_index(drop=True)
    else:
        return df.insert(0, column, None)


def stable_hash_meta(metadata) -> str:
    """Stable hash document based on its metadata."""
    return hashlib.sha1(json.dumps(metadata, sort_keys=True).encode()).hexdigest()

In [8]:
# create embeddings model and vector store
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

embeddings_model = AzureOpenAIEmbeddings(azure_deployment="Embedding")
docs_vectorstore = Chroma(
    collection_name="docs_store",
    embedding_function=embeddings_model,
    persist_directory="docs-db",
)

In [9]:
text = "this is a test document"
query_result = embeddings_model.embed_query(text)

In [11]:
query_result

[-0.012222584727053133,
 0.0072103982392216145,
 -0.014818063280923775,
 -0.026444746872933557,
 -0.0034330499700826883,
 0.024392201416343223,
 -0.010818907511946661,
 -0.004250758139302961,
 -0.01581123028712514,
 -0.024630560380244512,
 0.01402352874464015,
 0.027967604501313203,
 -0.01941312018494424,
 0.027755728624970902,
 -0.0012182858233069586,
 0.019214485852381433,
 0.015943653175500342,
 -0.011904770912539683,
 0.00400577654086686,
 -0.015506659879036249,
 0.0018572237218634153,
 0.00010242042755004594,
 -0.017771083074613947,
 0.008236671433178046,
 -0.018486163691607944,
 -0.0066409820543823165,
 0.0170957289516035,
 -0.033449890473363886,
 0.006339721100915767,
 -0.004969149607903097,
 0.001149591706076018,
 -0.013414386065817303,
 -0.00727660921774795,
 -0.0351184143965433,
 -0.004840037571134033,
 0.003764106026867532,
 -0.00983898204952479,
 -0.03866733439395429,
 0.024352474922359677,
 -0.0175194807042881,
 0.005551807802183156,
 0.00719053499222984,
 0.00260044436561

In [ ]:
# load documents with the LangChain document loader
from langchain_community.document_loaders import BSHTMLLoader, DirectoryLoader

loader = DirectoryLoader(
    "docs",
    glob="*.html",
    loader_cls=BSHTMLLoader,
    loader_kwargs={"open_encoding": "utf-8"},
    recursive=True,
    show_progress=True,
)
docs = loader.load()

In [ ]:
# divide documents into splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
splits = text_splitter.split_documents(docs)
splits_ids = [
    {"doc": split, "id": stable_hash_meta(split.metadata)} for split in splits
]

In [ ]:
# only keep splits that are not already in the vector store
existing_ids = docs_vectorstore.get()["ids"]
new_splits_ids = [split for split in splits_ids if split["id"] not in existing_ids]

In [ ]:
from langchain_core.documents import Document

# add new splits to the vector store
from langchain_core.documents import Document

if new_splits_ids:
    docs_vectorstore.add_documents(
        documents=[split["doc"] for split in new_splits_ids],
        ids=[split["id"] for split in new_splits_ids],
    )
docs_vectorstore.persist()

In [ ]:
import hashlib
import json
from langchain_core.documents import Document

def stable_hash_meta(doc: Document) -> str:
    """
    Stable hash document based on its metadata.
    """
    return hashlib.sha1(json.dumps(doc.metadata, sort_keys=True).encode()).hexdigest()

...
splits = text_splitter.split_documents(docs)
splits_ids = [
    {"doc": split, "id": stable_hash_meta(split.metadata)} for split in splits
]

existing_ids = docs_vectorstore.get()["ids"]
new_splits_ids = [split for split in splits_ids if split["id"] not in existing_ids]

docs_vectorstore.add_documents(
    documents=[split["doc"] for split in new_splits_ids],
    ids=[split["id"] for split in new_splits_ids],
)
docs_vectorstore.persist()

NameError: name 'text_splitter' is not defined